# Parameter estimation for qualitative data using the optimal scaling approach

This notebook explains, how to run parameter estimation with qualitative data as described in Schmiester et al. (2019), BioRxiv (https://www.biorxiv.org/content/10.1101/848648v1)

In [ ]:
import petab # works with petab v0.0.0a13
import pypesto
import amici

import numpy as np

## Load model

In [ ]:
folder_base = "./"
model_name = "Boehm_JProteomeRes2014OptimalScaling"

petab_problem = petab.Problem.from_folder(folder_base + model_name)

## Import model to AMICI

In [ ]:
importer = pypesto.PetabImporter(petab_problem)

model = importer.create_model()

## Create pyPESTO objective function

In [ ]:
hierarchicalProblem = pypesto.HierarchicalProblem.from_parameter_df(petab_problem.parameter_df)

# method can be set to 'reduced' or 'standard'. To use reparameterization, set 'reparameterized' = True.
# 'multistart' indicates, if a single local optimization or a multi-start optimization is run for the inner problem. Default is false, as problem is convex.
# 'weightsWithSurrogate' defines, if the weights used for the weighted least-squares function are dependend on the surrogate data.
# 'intervalConstraints' defines the minimal gaps and interval size. 'max' or 'max-min'
# 'timeInnerProblem' is used for analysis. if True, the inner problem is repeated several times and the mean computation time is returned.

hierarchicalProblem.optimalScalingOptions = {'method': 'reduced', 'multistart': False,
                                             'weightsWithSurrogate': False,
                                             'intervalConstraints': 'max',
                                             'reparameterized': True, 'timeInnerProblem': False,
                                             'modelName': model_name}

obj = importer.create_objective(hierarchical_problem=hierarchicalProblem)
# As gradient computation is not yet implemented, we set grad and hess to false:
obj.grad = False
obj.hess = False

## Run pyPESTO optimization

In [ ]:
optimizer = pypesto.ScipyOptimizer(method='Powell',
                                   options={'disp': True, 'maxiter': 500, 'maxfev': 500, 'ftol': 1e-10}})

problem = importer.create_problem(obj)
engine = pypesto.SingleCoreEngine()

results = pypesto.minimize(problem=problem, optimizer=optimizer,
                           n_starts=100, engine=engine)